In [14]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA

In [1]:
from langchain.document_loaders.url import UnstructuredURLLoader

URLs=[
    "https://www.potentiam.co.uk/"
]

# def DataLoader(URLs):
loader=UnstructuredURLLoader(urls=URLs)
data=loader.load()
data

[Document(page_content='OFFSHORING PLUS+\n\nUnlock your full\n\npotential with a\n\nglobal team\n\nWe help you scale your business in a smarter way by building your offshore team\n\nSpeak to an advisor\n\nWHAT IS OFFSHORING PLUS+\n\nThe step-change for your business ambition and your people\n\nPotentiam is much more than an offshoring provider. We are both a business consultancy and an offshore service provider - focused squarely on realising your ambitions and the power of your people.\n\nWe partner with your management team to create Offshore Strategies that :\n\nEnables 3-5 x faster profitable growth\n\nDelivers significant free growth capital\n\nUnlocks your most valuable human resources\n\nLowers the risk of building new capabilities\n\nHow it works\n\nTHE POTENTIAM PROCESS\n\nHow it works\n\nOur experts help you plan, structure, build, integrate and operate outstanding international teams that allow your business and your in-house people to grow.\n\nStrategic Options\n\nOur strat

In [4]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=500,
                                    chunk_overlap=50)

In [5]:
text_chunks=text_splitter.split_documents(data)

In [6]:
len(text_chunks)

13

In [7]:
text_chunks[0]

Document(page_content='OFFSHORING PLUS+\nUnlock your full\npotential with a\nglobal team\nWe help you scale your business in a smarter way by building your offshore team\nSpeak to an advisor\nWHAT IS OFFSHORING PLUS+\nThe step-change for your business ambition and your people\nPotentiam is much more than an offshoring provider. We are both a business consultancy and an offshore service provider - focused squarely on realising your ambitions and the power of your people.', metadata={'source': 'https://www.potentiam.co.uk/'})

In [8]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

f:\waste\Chatbot\chatapp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
f:\waste\Chatbot\chatapp\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Hp\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate develope

In [10]:
db = FAISS.from_documents(text_chunks, embeddings)
db.save_local("vectorstore")

In [11]:
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [13]:
prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])

In [16]:
llm = CTransformers(
        model = "F:\waste\Chatbot\llm\llama-2-7b-chat.ggmlv3.q8_0.bin",
        model_type="llama",
        max_new_tokens = 512,
        temperature = 0.5
    )

In [17]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': 2}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )

In [18]:
query='what is Potentiam?'

In [19]:
qa_chain({'query': query})

f:\waste\Chatbot\chatapp\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
